In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

In [ ]:
import numpy as np
from tqdm.autonotebook import tqdm

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
from cycle_gan.model.models import CycleGan
from cycle_gan.data.datasets import ImgDataset
from cycle_gan.utils import show_image, show_images, show_epoch_res

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
train_dataset = ImgDataset('../pytorch-CycleGAN-and-pix2pix/datasets/horse2zebra/', mode='train')
test_dataset = ImgDataset('../pytorch-CycleGAN-and-pix2pix/datasets/horse2zebra/', mode='test')

print(len(train_dataset), len(test_dataset))

In [ ]:
train_dataLoader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataLoader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
model = CycleGan(n_residual=6, 
                 device=device
                )

# Train

In [ ]:
for epoch in tqdm(range(100)):
    print('start epoch: {epoch}')
    
    for data in tqdm(train_dataLoader):
        model.train_step(data)    
        
    for data in test_dataLoader:
        pred =  model.test_step(data)
        break
    
    torch.save(model.state_dict(), f'checkpoints/CycleGan_epoch_{epoch:03d}.pth')       
    clear_output(wait=True)
    show_epoch_res(pred)  
    plt.savefig(f'val_img/{epoch:03d}.png', pad_inches=0, transparent=True)

# Test

In [ ]:
for idx, data in enumerate(test_dataLoader):
    pred =  model.test_step(data)
    show_epoch_res(pred)
    plt.savefig(f'processed_img/{idx:03d}.png', pad_inches=0, transparent=True)

Часть изображений трансформированны достаточно хорошо, часть с артифактами
В целом, можно заметить тенденцию, что чем более "чисто" изображение лошади\зебро, тем лучше происходит наложение стиля